In [1]:
import numba as nb
print(f"Version of Numba: {nb.__version__}")

Version of Numba: 0.55.1


In [2]:
import itertools
import numpy as np
import numba


@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_sparse_loop(input, indices, offsets,
                           values, output):
    for i, j in enumerate(indices):
        for off, val in zip(offsets, values):
            output[i] += input[j + off] * val


def correlate_sparse(image, kernel):
    indices = np.nonzero(kernel)
    offsets = np.ravel_multi_index(indices, image.shape)
    values = kernel[indices].astype(image.dtype)
    result = np.zeros([a - b + 1
                       for a, b in zip(image.shape, kernel.shape)],
                      dtype=image.dtype)
    corner_multi_indices = np.meshgrid(*[np.arange(i)
                                         for i in result.shape],
                                       indexing='ij',
                                       sparse=True)
    corner_indices = np.ravel_multi_index(corner_multi_indices,
                                          image.shape).ravel()
    _correlate_sparse_loop(
        image.ravel(), corner_indices, offsets, values,
        result.ravel()
    )
    return result

In [3]:
image = np.random.random((4000, 6000))
w = 301
kern = np.zeros((w + 1,) * image.ndim)
for indices in itertools.product(*([[0, -1]] * image.ndim)):
    kern[indices] = (-1) ** (image.ndim % 2 != np.sum(indices) % 2)

In [4]:
indices = np.nonzero(kern)
offsets = np.ravel_multi_index(indices, image.shape)
values = kern[indices].astype(image.dtype)
result = np.zeros([a - b + 1
                   for a, b in zip(image.shape, kern.shape)],
                  dtype=image.dtype)
corner_multi_indices = np.meshgrid(*[np.arange(i)
                                     for i in result.shape],
                                   indexing='ij',
                                   sparse=True)
corner_indices = np.ravel_multi_index(corner_multi_indices,
                                      image.shape).ravel()

In [5]:
_correlate_sparse_loop(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [6]:
%%timeit -n 5 -r 5 -o
_correlate_sparse_loop(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

122 ms ± 13.3 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


<TimeitResult : 122 ms ± 13.3 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)>

In [7]:
%%timeit -n 1 -r 1 -o
_correlate_sparse_loop.py_func(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

50.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 50.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [8]:
@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_sparse_offsets(input, indices, offsets, values, output):
    for off, val in zip(offsets, values):
        # this loop order optimises cache access, gives up to 10x speedup
        for i, j in enumerate(indices):
            output[i] += input[j + off] * val 

In [9]:
_correlate_sparse_offsets(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [10]:
%%timeit -n 5 -r 5
_correlate_sparse_offsets(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

150 ms ± 18.2 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [11]:
def print_lines(substring, string):
    lines = string.split('\n')
    for line in lines:
        if substring in line:
            print(line)

In [12]:
asm = list(_correlate_sparse_offsets.inspect_asm().values())[0]
print('single instructions:', asm.count('sd'))
print('packed instructions:', asm.count('pd'))
print_lines('sd', asm)

single instructions: 10
packed instructions: 0
	vmovsd	(%rbx,%rdx), %xmm0
	vmovsd	(%rcx,%rbx,8), %xmm1
	vfmadd213sd	(%rsi,%r9,8), %xmm0, %xmm1
	vmovsd	%xmm1, (%rsi,%r9,8)
	vmovsd	(%rcx,%rbx,8), %xmm1
	vfmadd213sd	8(%rsi,%r9,8), %xmm0, %xmm1
	vmovsd	%xmm1, 8(%rsi,%r9,8)
	vmovsd	(%rcx,%rdx,8), %xmm1
	vfmadd213sd	(%rsi,%r11,8), %xmm1, %xmm0
	vmovsd	%xmm0, (%rsi,%r11,8)


In [13]:
@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_no_indirection(input, indices, offsets, values, output):
    for off, val in zip(offsets, values):
        for i in range(len(indices)):
            output[i] += input[i + off] * val

In [14]:
# warmup jit
_correlate_no_indirection(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [15]:
%%timeit -n 5 -r 5
_correlate_no_indirection(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

149 ms ± 16.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [16]:
asm = list(_correlate_no_indirection.inspect_asm().values())[0]
print('single instructions:', asm.count('sd'))
print('packed instructions:', asm.count('pd'))
print_lines('pd', asm)

single instructions: 10
packed instructions: 0


In [17]:
@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_no_indir_offset(input, indices, offsets, values, output):
    for off, val in zip(offsets, values):
        for i in range(len(indices)):
            output[i] += input[i] * val

In [18]:
_correlate_no_indir_offset(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [19]:
%%timeit -n 5 -r 5
_correlate_no_indir_offset(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

95.1 ms ± 10.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [20]:
asm = list(_correlate_no_indir_offset.inspect_asm().values())[0]
print('single instructions:', asm.count('sd'))
print('packed instructions:', asm.count('pd'))
print_lines('pd', asm)

single instructions: 16
packed instructions: 0


In [21]:
image32 = image.astype(np.float32)
result32 = result.astype(np.float32)
values32 = values.astype(np.float32)

In [22]:
_correlate_sparse_offsets(
    image32.ravel(), corner_indices, offsets, values32,
    result32.ravel()
)

In [23]:
%%timeit -n 5 -r 5
_correlate_sparse_offsets(
    image32.ravel(), corner_indices, offsets, values32,
    result32.ravel()
)

120 ms ± 16.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [24]:
%%timeit -n 5 -r 5
_correlate_no_indir_offset(
    image32.ravel(), corner_indices, offsets, values32,
    result32.ravel()
)

68.4 ms ± 7.41 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [25]:
asm = list(_correlate_no_indir_offset.inspect_asm().values())[1]
print('single instructions:', asm.count('ss'))
print('packed instructions:', asm.count('ps'))
print_lines('ps', asm)

single instructions: 21
packed instructions: 50
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 672(%rsp)
	vmovups	%ymm0, 696(%rsp)
	vmovaps	%ymm0, 608(%rsp)
	vmovups	%ymm0, 632(%rsp)
	vmovaps	%ymm0, 544(%rsp)
	vmovups	%ymm0, 568(%rsp)
	vmovaps	%ymm0, 480(%rsp)
	vmovups	%ymm0, 504(%rsp)
	vmovaps	%ymm0, 736(%rsp)
	vmovups	%ymm0, 760(%rsp)
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 672(%rsp)
	vmovups	%ymm0, 696(%rsp)
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 608(%rsp)
	vmovups	%ymm0, 632(%rsp)
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 544(%rsp)
	vmovups	%ymm0, 568(%rsp)
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 480(%rsp)
	vmovups	%ymm0, 504(%rsp)
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 736(%rsp)
	vmovups	%ymm0, 760(%rsp)
	vmovups	744(%rsp), %xmm0
	vmovaps	%xmm0, 816(%rsp)
	vmovaps	768(%rsp), %xmm0
	vmovaps	%xmm0, 800(%rsp)
	vmovaps	800(%rsp), %xmm0
	vmovups	%xmm0, 224(%rsp)
	vmovaps	816(%rsp), %xmm0
	vmovups	%xmm0, 200(%rsp)
	vmovups	296(%rsp), %ymm0
	vmovups	328(%rsp), %xmm1
	vmo